## Credible intervals from model runs
Plot credible intervals for candidate models and for selected primary analysis.

In [ ]:
# Uncomment the following commands to install in Colab
# Once completed, click the 'Restart runtime' button that will appear to restart the Colab environment
# and then proceed to the following cells

# !pip uninstall numba -y
# !pip uninstall librosa -y
# !pip install estival==0.4.7b2 numpy==1.24.3 kaleido

In [ ]:
try:
    import google.colab
    on_colab = True
    ! git clone https://github.com/monash-emu/aust-covid.git --branch finalise-notebooks
    %cd aust-covid
    %pip install -e ./
    import multiprocessing as mp
    mp.set_start_method('forkserver')
except:
    on_colab = False

In [ ]:
import pandas as pd
from emutools.tex import DummyTexDoc, StandardTexDoc, add_image_to_doc
from aust_covid.calibration import get_targets
from inputs.constants import RUN_IDS, RUNS_PATH, SUPPLEMENT_PATH, PRIMARY_ANALYSIS
from estival.sampling import tools as esamp
from emutools.calibration import plot_output_ranges, plot_output_ranges_by_analysis

In [ ]:
quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]
spaghettis = {k: pd.read_hdf(RUNS_PATH / v / 'output/results.hdf', 'spaghetti') for k, v in RUN_IDS.items()}
quantile_outputs = {k: esamp.quantiles_for_results(spaghettis[k], quantiles) for k in RUN_IDS.keys()}

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'ranges', 'Output ranges', 'austcovid')

In [ ]:
targets = get_targets(DummyTexDoc())

In [ ]:
case_ranges_by_analysis = plot_output_ranges_by_analysis(quantile_outputs, targets, 'notifications_ma', quantiles)
title = 'Cases credible intervals by analysis.'
caption_end = ' with median estimate (black line), 2.5 to 97.5 centile credible interval ' \
    '(light blue shading), and 25 to 75 centile credible interval (dark blue shading), ' \
    'with comparison against epidemiological targets (red circles). ' \
    'Panel for each of the four candidate analyses. '
caption = f'Case notifications {caption_end}'
add_image_to_doc(case_ranges_by_analysis, 'case_ranges', 'svg', title, app_doc, 'Analysis comparison', caption=caption)
case_ranges_by_analysis

In [ ]:
death_ranges_by_analysis = plot_output_ranges_by_analysis(quantile_outputs, targets, 'deaths_ma', quantiles)
caption = f'Deaths {caption_end}'
add_image_to_doc(death_ranges_by_analysis, 'death_ranges', 'svg', 'Deaths credible intervals by analysis.', app_doc, 'Analysis comparison', caption=caption)
death_ranges_by_analysis

In [ ]:
seropos_ranges_by_analysis = plot_output_ranges_by_analysis(quantile_outputs, targets, 'adult_seropos_prop', quantiles)
caption = f'Seropositive proportion {caption_end}'
add_image_to_doc(seropos_ranges_by_analysis, 'seropos_ranges', 'svg', 'Seropositive proportion credible intervals by analysis.', app_doc, 'Analysis comparison', caption=caption)
seropos_ranges_by_analysis

In [ ]:
outputs = ['notifications_ma', 'deaths_ma', 'adult_seropos_prop', 'reproduction_number']
base_analysis_ranges = plot_output_ranges(quantile_outputs, targets, outputs, PRIMARY_ANALYSIS, quantiles)
caption = 'Primary analysis model median estimate (black line), 2.5 to 97.5 centile credible interval ' \
    '(light blue shading), and 25 to 75 centile credible interval (dark blue shading), ' \
    'with comparison against epidemiological targets (red circles). ' \
    'Panel for each epidemiological output. '
add_image_to_doc(base_analysis_ranges, 'base_ranges', 'svg', 'Base analysis output credible intervals.', app_doc, 'Calibration results', caption=caption)
app_doc.write_doc()
base_analysis_ranges

In [ ]:
if on_colab:
    # To build a PDF, we need the appropriate tex packages installed
    ! apt-get -y install texlive-latex-base texlive-fonts-recommended texlive-fonts-extra texlive-latex-extra texlive-bibtex-extra biber
    # To avoid clutter and navigate to the right directory
    import os
    os.chdir(SUPPLEMENT_PATH)
    # And finally build the formatted PDF, repeated commands are necessary
    ! pdflatex ranges
    ! biber ranges
    ! pdflatex ranges
    ! pdflatex ranges